The 'ml' service should be up and running waiting to consume messages from the KAFKA_CONSUMER_TOPIC set in the docker-compose.yaml file for the 'ml' service. Run the bash script below and see the outputs.


In [ ]:
# jupyterlab_vim

In [ ]:
create topic for ml service to publish to

In [ ]:
! python /home/app/kafka/src/create_topic.py --topic=book_emotions

In [ ]:
# create and publish source topic for ml service to ingest from

In [ ]:
! python /home/app/kafka/src/create_topic.py --topic=book --file=/home/app/data/brothers-karamazov.txt --format=json

In [ ]:
## output of ml inference
from kafka import KafkaConsumer
from kafka.admin import KafkaAdminClient

bootstrap_servers = "kafka-broker:9092"

admin_client = KafkaAdminClient(
    bootstrap_servers=bootstrap_servers,
    client_id="demo",
)

In [ ]:
producer_topic = os.getenv("KAFKA_PRODUCER_TOPIC", "book_emotions")
producer_topic = KafkaConsumer(
    producer_topic,
    bootstrap_servers=["kafka-broker:9092"],
    auto_offset_reset="earliest",
    enable_auto_commit=False,
)

result_batch = producer_topic.poll(timeout_ms=1000)
for partition, messages in result_batch.items():
    for message in messages:
        print(message.value.decode("utf-8"))
producer_topic.close()

In [ ]:
# ingest this new topic into rising wave for further analysis

In [ ]:
! export PYTHONPATH=../risingwave:$PYTHONPATH && python ../risingwave/src/main.py --topic='book_emotions' --bootstrap-servers='kafka-broker:9092' --schema='{"emotion": "VARCHAR", "text": "VARCHAR", "line": "INTEGER"}' --source=kafka

In [ ]:
import psycopg2
import pandas as pd

conn = psycopg2.connect(host="risingwave", port=4566, user="root", dbname="dev")
conn.autocommit = True

with conn.cursor() as cur:
    cur.execute(
        "SELECT kafka_timestamp, line, text, emotion FROM book_emotions LIMIT 10;"
    )
    values = cur.fetchall()

In [ ]:
df = pd.DataFrame(values, columns=["timestamp", "line", "text", "emotion"])
print(df.tail())

In [ ]:
with conn.cursor() as cur:
    cur.execute(
        "SELECT emotion, count(*) FROM book_emotions group by emotion order by emotion;"
    )
    values = cur.fetchall()
summary = pd.DataFrame(values, columns=["emotion", "count"])

In [ ]:
summary["count"].sum()